In [70]:
import numpy as np

coords = np.array([[-1.8, -1, 0.2, 1.2, 2], [5.69, 1.99, 1.24, 9.81, 14.5]])

coords_transposed = coords.T
print(coords_transposed)

[[-1.8   5.69]
 [-1.    1.99]
 [ 0.2   1.24]
 [ 1.2   9.81]
 [ 2.   14.5 ]]


In [71]:
m = 1
xi = coords_transposed[:, 0]
yi = coords_transposed[:, 1]
s = np.zeros((1, m*2+1))

for x in xi:
    temp = np.array([])
    for k in range(0, m*2+1):
        temp = np.append(temp, x**k)
    s = np.vstack((s, temp))


sk = np.sum(s, axis=0)
s = np.vstack((s, sk))
print(s)

[[ 0.    0.    0.  ]
 [ 1.   -1.8   3.24]
 [ 1.   -1.    1.  ]
 [ 1.    0.2   0.04]
 [ 1.    1.2   1.44]
 [ 1.    2.    4.  ]
 [ 5.    0.6   9.72]]


In [72]:
# tworzymy wstępną macierz wyników i nadajemy jej kształt, wypełniając zerami
wyn = np.zeros((1, m+1))

# uruchamiamy pętle m+1 razy, gdyż taka będzie liczba kolumn w macierzy wyników
for i in range(m+1):
    # tworzymy tymczasową macierz, która będzie zawierała kolejne elementy wiersza
    temp = np.array([])
    # uruchamiamy pętle m+1 razy, gdyż taka będzie liczba elementów w wierszu
    for j in range(m+1):
        # dodajemy kolejne elementy do tymczasowej macierzy
        temp = np.append(temp, sk[i+j])
    # dodajemy wiersz do macierzy wyników
    wyn = np.vstack((wyn, temp))

# usuwamy pierwszy wiersz, który zawierał tylko zera
wyn = np.delete(wyn, 0, 0)
print(wyn)

[[5.   0.6 ]
 [0.6  9.72]]
